# Infer-6-TrueSkill : Systeme de Classement et Apprentissage en Ligne

**Serie** : Programmation Probabiliste avec Infer.NET (6/12)  
**Duree estimee** : 55 minutes  
**Prerequis** : Infer-5-Skills-IRT

---

## Objectifs

- Comprendre le systeme TrueSkill (Xbox Live)
- Implementer des matchs 1v1 et la mise a jour des skills
- Gerer les matchs nuls
- Maitriser l'apprentissage en ligne (posterieurs -> priors)
- Etendre aux equipes et multi-joueurs

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-5-Skills-IRT](Infer-5-Skills-IRT.ipynb) | [Infer-7-Classification](Infer-7-Classification.ipynb) |

---

## 1. Configuration

Nous chargeons Infer.NET pour implementer le systeme TrueSkill, developpe par Microsoft Research pour Xbox Live. Ce systeme de classement bayesien estime les competences des joueurs a partir des resultats de matchs, tout en quantifiant l'incertitude sur ces estimations.

In [1]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


## 2. Introduction a TrueSkill

### Contexte

**TrueSkill** est le systeme de classement developpe par Microsoft Research pour Xbox Live. Il est une generalisation bayesienne du systeme Elo.

### Principe

Chaque joueur a un **skill** (competence) modelise par une distribution Gaussienne :
- **Moyenne (mu)** : estimation du skill
- **Variance (sigma^2)** : incertitude sur cette estimation

### Formulation

$$\text{skill}_i \sim \mathcal{N}(\mu_i, \sigma_i^2)$$

$$\text{performance}_i = \text{skill}_i + \epsilon_i, \quad \epsilon_i \sim \mathcal{N}(0, \beta^2)$$

$$\text{Joueur 1 gagne si} \quad \text{performance}_1 > \text{performance}_2$$

### Parametres par defaut

| Parametre | Valeur | Description |
|-----------|--------|-------------|
| mu_initial | 25 | Skill initial |
| sigma_initial | 25/3 | Incertitude initiale |
| beta | sigma/2 | Ecart-type de la performance |

## 3. Modele Deux Joueurs

In [2]:
// Parametres TrueSkill
double muInitial = 25.0;
double sigmaInitial = 25.0 / 3.0;
double beta = sigmaInitial / 2.0;  // Variabilite de la performance

// Skills des joueurs (priors)
Variable<double> skill1 = Variable.GaussianFromMeanAndVariance(muInitial, sigmaInitial * sigmaInitial).Named("skill1");
Variable<double> skill2 = Variable.GaussianFromMeanAndVariance(muInitial, sigmaInitial * sigmaInitial).Named("skill2");

// Performances (skill + bruit)
Variable<double> perf1 = Variable.GaussianFromMeanAndVariance(skill1, beta * beta).Named("perf1");
Variable<double> perf2 = Variable.GaussianFromMeanAndVariance(skill2, beta * beta).Named("perf2");

// Resultat du match : Joueur 1 gagne
Variable<bool> joueur1Gagne = (perf1 > perf2).Named("joueur1Gagne");

// Observation : Joueur 1 a effectivement gagne
joueur1Gagne.ObservedValue = true;

Console.WriteLine("Modele TrueSkill deux joueurs defini.");

Modele TrueSkill deux joueurs defini.


In [3]:
// Inference apres le match
InferenceEngine moteur = new InferenceEngine(new ExpectationPropagation());
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Gaussian skill1Post = moteur.Infer<Gaussian>(skill1);
Gaussian skill2Post = moteur.Infer<Gaussian>(skill2);

Console.WriteLine("=== Apres un match (Joueur 1 gagne) ===");
Console.WriteLine($"\nAvant le match :");
Console.WriteLine($"  Skill1 = N({muInitial:F1}, {sigmaInitial:F2})");
Console.WriteLine($"  Skill2 = N({muInitial:F1}, {sigmaInitial:F2})");

Console.WriteLine($"\nApres le match :");
Console.WriteLine($"  Skill1 = N({skill1Post.GetMean():F2}, {Math.Sqrt(skill1Post.GetVariance()):F2})");
Console.WriteLine($"  Skill2 = N({skill2Post.GetMean():F2}, {Math.Sqrt(skill2Post.GetVariance()):F2})");

Console.WriteLine($"\nChangement de skill :");
Console.WriteLine($"  Joueur 1 : +{skill1Post.GetMean() - muInitial:F2}");
Console.WriteLine($"  Joueur 2 : {skill2Post.GetMean() - muInitial:F2}");

Compiling model...

done.


=== Apres un match (Joueur 1 gagne) ===



Avant le match :


  Skill1 = N(25,0, 8,33)


...

  Skill1 = N(29,21, 7,19)


  Skill2 = N(20,79, 7,19)



Changement de skill :


  Joueur 1 : +4,21


  Joueur 2 : -4,21


### Analyse

- Le **gagnant** voit son skill augmenter
- Le **perdant** voit son skill diminuer
- L'**incertitude** diminue pour les deux (on a plus d'information)
- Le changement est symetrique si les priors sont identiques

## 4. Gestion des Matchs Nuls

### Modele

Un match nul se produit quand la difference de performances est dans un intervalle $[-\epsilon, \epsilon]$.

$$|\text{perf}_1 - \text{perf}_2| < \epsilon \Rightarrow \text{match nul}$$

In [4]:
// Modele avec possibilite de match nul

double epsilon = 1.0;  // Marge pour match nul

Variable<double> skillA = Variable.GaussianFromMeanAndVariance(muInitial, sigmaInitial * sigmaInitial).Named("skillA");
Variable<double> skillB = Variable.GaussianFromMeanAndVariance(muInitial, sigmaInitial * sigmaInitial).Named("skillB");

Variable<double> perfA = Variable.GaussianFromMeanAndVariance(skillA, beta * beta);
Variable<double> perfB = Variable.GaussianFromMeanAndVariance(skillB, beta * beta);

// Difference de performances
Variable<double> diff = perfA - perfB;

// Match nul : diff dans [-epsilon, epsilon]
Variable.ConstrainBetween(diff, -epsilon, epsilon);

InferenceEngine moteurNul = new InferenceEngine(new ExpectationPropagation());
moteurNul.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Gaussian skillAPostNul = moteurNul.Infer<Gaussian>(skillA);
Gaussian skillBPostNul = moteurNul.Infer<Gaussian>(skillB);

Console.WriteLine("=== Apres un match nul ===");
Console.WriteLine($"\nSkillA = N({skillAPostNul.GetMean():F2}, {Math.Sqrt(skillAPostNul.GetVariance()):F2})");
Console.WriteLine($"SkillB = N({skillBPostNul.GetMean():F2}, {Math.Sqrt(skillBPostNul.GetVariance()):F2})");

Console.WriteLine($"\n=> Les deux joueurs gardent le meme skill moyen");
Console.WriteLine($"   mais l'incertitude diminue (on sait qu'ils sont proches)");

Compiling model...

done.


=== Apres un match nul ===



SkillA = N(25,00, 6,46)


SkillB = N(25,00, 6,46)



=> Les deux joueurs gardent le meme skill moyen


   mais l'incertitude diminue (on sait qu'ils sont proches)


## 5. Apprentissage en Ligne

### Principe

Apres chaque match, les **posterieurs** deviennent les **priors** pour le match suivant.

```
Match 1 : Prior -> Inference -> Posterieur
                                    |
                                    v
Match 2 : Prior (= Posterieur 1) -> Inference -> Posterieur
                                                     |
                                                     v
Match 3 : ...
```

In [5]:
// Classe pour gerer l'apprentissage en ligne

public class TrueSkillOnline
{
    private double muInit;
    private double sigmaInit;
    private double beta;
    private InferenceEngine moteur;
    
    // Skills actuels des joueurs
    private Dictionary<string, Gaussian> skills;
    
    public TrueSkillOnline(double muInit = 25, double sigmaInit = 8.33, double beta = 4.17)
    {
        this.muInit = muInit;
        this.sigmaInit = sigmaInit;
        this.beta = beta;
        this.skills = new Dictionary<string, Gaussian>();
        this.moteur = new InferenceEngine(new ExpectationPropagation());
        this.moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    }
    
    public Gaussian GetSkill(string joueur)
    {
        if (!skills.ContainsKey(joueur))
        {
            skills[joueur] = Gaussian.FromMeanAndVariance(muInit, sigmaInit * sigmaInit);
        }
        return skills[joueur];
    }
    
    public void EnregistrerMatch(string gagnant, string perdant)
    {
        // Priors actuels
        Gaussian priorGagnant = GetSkill(gagnant);
        Gaussian priorPerdant = GetSkill(perdant);
        
        // Modele
        Variable<Gaussian> priorG = Variable.Observed(priorGagnant);
        Variable<Gaussian> priorP = Variable.Observed(priorPerdant);
        
        Variable<double> skillG = Variable.Random<double, Gaussian>(priorG);
        Variable<double> skillP = Variable.Random<double, Gaussian>(priorP);
        
        Variable<double> perfG = Variable.GaussianFromMeanAndVariance(skillG, beta * beta);
        Variable<double> perfP = Variable.GaussianFromMeanAndVariance(skillP, beta * beta);
        
        Variable<bool> resultat = (perfG > perfP);
        resultat.ObservedValue = true;  // Le gagnant a gagne
        
        // Inference
        skills[gagnant] = moteur.Infer<Gaussian>(skillG);
        skills[perdant] = moteur.Infer<Gaussian>(skillP);
    }
    
    public void AfficherClassement()
    {
        var classement = skills.OrderByDescending(kv => kv.Value.GetMean());
        Console.WriteLine("\n=== Classement ===");
        int rang = 1;
        foreach (var kv in classement)
        {
            double mu = kv.Value.GetMean();
            double sigma = Math.Sqrt(kv.Value.GetVariance());
            double conservatif = mu - 3 * sigma;  // TrueSkill rating
            Console.WriteLine($"{rang}. {kv.Key,-10} : mu={mu:F1}, sigma={sigma:F2}, rating={conservatif:F1}");
            rang++;
        }
    }
}

Console.WriteLine("Classe TrueSkillOnline definie.");

Classe TrueSkillOnline definie.


In [6]:
// Simulation d'un tournoi

var ts = new TrueSkillOnline();

Console.WriteLine("=== Tournoi TrueSkill ===");

// Serie de matchs
var matchs = new (string, string)[] {
    ("Alice", "Bob"),     // Alice bat Bob
    ("Charlie", "Dave"),  // Charlie bat Dave
    ("Alice", "Charlie"), // Alice bat Charlie
    ("Bob", "Dave"),      // Bob bat Dave
    ("Alice", "Dave"),    // Alice bat Dave
    ("Charlie", "Bob")    // Charlie bat Bob
};

foreach (var (gagnant, perdant) in matchs)
{
    Console.WriteLine($"Match : {gagnant} bat {perdant}");
    ts.EnregistrerMatch(gagnant, perdant);
}

ts.AfficherClassement();

=== Tournoi TrueSkill ===


Match : Alice bat Bob


Compiling model...

done.


Match : Charlie bat Dave


...


=== Classement ===


1. Alice      : mu=33,3, sigma=6,01, rating=15,2


2. Charlie    : mu=28,3, sigma=5,58, rating=11,6


3. Bob        : mu=21,7, sigma=5,58, rating=4,9


4. Dave       : mu=16,7, sigma=6,01, rating=-1,3


### Analyse du rating

Le **rating conservatif** = mu - 3*sigma est utilise pour le classement public.
Il represente le skill minimum avec 99% de confiance.

- **Alice** est clairement en tete (3 victoires, 0 defaite)
- **Charlie** est deuxieme (2 victoires, 1 defaite)
- Les ratings augmentent avec les victoires et la confiance

## 6. Extension aux Equipes

### Modele

Pour un match par equipes, la performance d'equipe est la somme des performances individuelles.

In [7]:
// Modele par equipes (2v2)

// Equipe 1 : Joueurs A et B
Variable<double> skillA2 = Variable.GaussianFromMeanAndVariance(25, 70);
Variable<double> skillB2 = Variable.GaussianFromMeanAndVariance(25, 70);

// Equipe 2 : Joueurs C et D
Variable<double> skillC = Variable.GaussianFromMeanAndVariance(25, 70);
Variable<double> skillD = Variable.GaussianFromMeanAndVariance(25, 70);

// Performances individuelles
Variable<double> perfA2 = Variable.GaussianFromMeanAndVariance(skillA2, 17);
Variable<double> perfB2 = Variable.GaussianFromMeanAndVariance(skillB2, 17);
Variable<double> perfC2 = Variable.GaussianFromMeanAndVariance(skillC, 17);
Variable<double> perfD2 = Variable.GaussianFromMeanAndVariance(skillD, 17);

// Performances d'equipe (somme)
Variable<double> perfEquipe1 = perfA2 + perfB2;
Variable<double> perfEquipe2 = perfC2 + perfD2;

// Equipe 1 gagne
Variable<bool> equipe1Gagne = (perfEquipe1 > perfEquipe2);
equipe1Gagne.ObservedValue = true;

InferenceEngine moteurEquipe = new InferenceEngine(new ExpectationPropagation());
moteurEquipe.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== Match par equipes (2v2) ===");
Console.WriteLine("Equipe 1 (A+B) bat Equipe 2 (C+D)\n");

Console.WriteLine($"Skill A apres : {moteurEquipe.Infer<Gaussian>(skillA2).GetMean():F2}");
Console.WriteLine($"Skill B apres : {moteurEquipe.Infer<Gaussian>(skillB2).GetMean():F2}");
Console.WriteLine($"Skill C apres : {moteurEquipe.Infer<Gaussian>(skillC).GetMean():F2}");
Console.WriteLine($"Skill D apres : {moteurEquipe.Infer<Gaussian>(skillD).GetMean():F2}");

Console.WriteLine("\n=> Tous les membres de l'equipe gagnante voient leur skill augmenter");

=== Match par equipes (2v2) ===


Equipe 1 (A+B) bat Equipe 2 (C+D)



Compiling model...

done.


Skill A apres : 27,99


Skill B apres : 27,99


Skill C apres : 22,01


Skill D apres : 22,01



=> Tous les membres de l'equipe gagnante voient leur skill augmenter


## 7. Multi-joueurs (Free-for-all)

### Modele

Pour N joueurs, on decompose le resultat en N-1 comparaisons par paires :
- 1er > 2e > 3e > ... > Ne

In [8]:
// Modele multi-joueurs (4 joueurs)
// Resultat : P1 > P2 > P3 > P4

Variable<double>[] skillsMulti = new Variable<double>[4];
Variable<double>[] perfsMulti = new Variable<double>[4];

for (int i = 0; i < 4; i++)
{
    skillsMulti[i] = Variable.GaussianFromMeanAndVariance(25, 70).Named($"skill{i+1}");
    perfsMulti[i] = Variable.GaussianFromMeanAndVariance(skillsMulti[i], 17).Named($"perf{i+1}");
}

// Contraintes d'ordre : perf1 > perf2 > perf3 > perf4
Variable.ConstrainTrue(perfsMulti[0] > perfsMulti[1]);
Variable.ConstrainTrue(perfsMulti[1] > perfsMulti[2]);
Variable.ConstrainTrue(perfsMulti[2] > perfsMulti[3]);

InferenceEngine moteurMulti = new InferenceEngine(new ExpectationPropagation());
moteurMulti.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine("=== Course multi-joueurs ===");
Console.WriteLine("Classement : P1 > P2 > P3 > P4\n");

for (int i = 0; i < 4; i++)
{
    Gaussian post = moteurMulti.Infer<Gaussian>(skillsMulti[i]);
    Console.WriteLine($"Joueur {i+1} (position {i+1}) : mu={post.GetMean():F2}, sigma={Math.Sqrt(post.GetVariance()):F2}");
}

=== Course multi-joueurs ===


Classement : P1 > P2 > P3 > P4



Compiling model...

done.


Iterating: 


...

 50


Joueur 1 (position 1) : mu=32,73, sigma=6,42


Joueur 2 (position 2) : mu=27,23, sigma=5,83


Joueur 3 (position 3) : mu=22,77, sigma=5,83


Joueur 4 (position 4) : mu=17,27, sigma=6,42


## 8. Analyse d'Echecs (Elo Bayesien)

In [9]:
// Simulation de parties d'echecs

var chess = new TrueSkillOnline(muInit: 1500, sigmaInit: 350, beta: 175);

// Donnees historiques simulees
var partiesEchecs = new (string, string)[] {
    ("Magnus", "Fabiano"),
    ("Magnus", "Ian"),
    ("Fabiano", "Ian"),
    ("Magnus", "Fabiano"),
    ("Magnus", "Ian"),
    ("Ian", "Fabiano"),  // Upset!
    ("Magnus", "Fabiano"),
    ("Magnus", "Ian"),
    ("Fabiano", "Ian"),
    ("Magnus", "Ian")
};

Console.WriteLine("=== Classement Echecs (Elo Bayesien) ===");
Console.WriteLine("\nParties :");
foreach (var (g, p) in partiesEchecs)
{
    Console.WriteLine($"  {g} bat {p}");
    chess.EnregistrerMatch(g, p);
}

chess.AfficherClassement();

=== Classement Echecs (Elo Bayesien) ===



Parties :


  Magnus bat Fabiano


Compiling model...

done.


...

done.



=== Classement ===


1. Magnus     : mu=1923,7, sigma=213,54, rating=1283,1


2. Fabiano    : mu=1344,7, sigma=183,39, rating=794,6


3. Ian        : mu=1217,4, sigma=182,68, rating=669,4


## 9. Exercice : Simuler un Tournoi

### Enonce

Creez un tournoi avec 6 joueurs et simulez 15 matchs aleatoires.
Comparez le classement final aux "vrais skills" que vous aurez definis.

### Indice

- Definissez des skills "vrais" pour chaque joueur
- Simulez le resultat de chaque match en fonction des skills
- Utilisez TrueSkillOnline pour mettre a jour les estimations

In [10]:
// EXERCICE : Tournoi simule

// Vrais skills (inconnus du systeme)
var vraisSkills = new Dictionary<string, double>
{
    ["Elite1"] = 35,
    ["Elite2"] = 32,
    ["Moyen1"] = 25,
    ["Moyen2"] = 24,
    ["Debutant1"] = 18,
    ["Debutant2"] = 15
};

var joueurs = vraisSkills.Keys.ToArray();
var rng = new Random(42);
var tournoi = new TrueSkillOnline();

Console.WriteLine("=== Tournoi Simule ===");
Console.WriteLine("\nVrais skills :");
foreach (var kv in vraisSkills.OrderByDescending(x => x.Value))
    Console.WriteLine($"  {kv.Key}: {kv.Value}");

Console.WriteLine("\nMatchs :");

// 15 matchs aleatoires
for (int m = 0; m < 15; m++)
{
    // Choisir deux joueurs differents
    int i = rng.Next(joueurs.Length);
    int j;
    do { j = rng.Next(joueurs.Length); } while (j == i);
    
    string j1 = joueurs[i];
    string j2 = joueurs[j];
    
    // Simuler le match (le meilleur gagne avec plus de probabilite)
    double perf1 = vraisSkills[j1] + rng.NextDouble() * 10 - 5;  // +/- 5
    double perf2 = vraisSkills[j2] + rng.NextDouble() * 10 - 5;
    
    string gagnant = perf1 > perf2 ? j1 : j2;
    string perdant = perf1 > perf2 ? j2 : j1;
    
    Console.WriteLine($"  {gagnant} bat {perdant}");
    tournoi.EnregistrerMatch(gagnant, perdant);
}

tournoi.AfficherClassement();

Console.WriteLine("\n=> Comparez le classement estime aux vrais skills !");

=== Tournoi Simule ===



Vrais skills :


  Elite1: 35


  Elite2: 32


  Moyen1: 25


...

2. Elite2     : mu=33,8, sigma=5,48, rating=17,4


3. Moyen2     : mu=24,1, sigma=5,44, rating=7,8


4. Moyen1     : mu=22,4, sigma=5,92, rating=4,6


5. Debutant1  : mu=12,6, sigma=4,48, rating=-0,8



=> Comparez le classement estime aux vrais skills !


## 10. Resume

| Concept | Description |
|---------|-------------|
| **TrueSkill** | Systeme de classement bayesien |
| **Skill** | Gaussienne N(mu, sigma^2) |
| **Performance** | Skill + bruit gaussien |
| **Match nul** | Difference de perf dans [-epsilon, epsilon] |
| **Online learning** | Posterieurs -> Priors |
| **Rating conservatif** | mu - 3*sigma |

---

## Prochaine etape

Dans [Infer-7-Classification](Infer-7-Classification.ipynb), nous explorerons :

- La classification bayesienne
- Le Bayes Point Machine
- Les tests cliniques A/B bayesiens